# Configuration

NOTES: The warnings after the import are referred to the fact that Tensorflow 2.x versions are built to directly look for a GPU in the system. The warning can be forgot if you are not going to use the GPU. 

In [1]:
!source myenv/bin/activate

In [2]:
LENGTH_CHOSEN = 126520

In [3]:
import os
import librosa
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set_style('whitegrid')
import IPython.display as ipd
import librosa.display
import numpy as np
import pickle
import scipy
import ipywidgets
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# from livelossplot import PlotLossesKeras
tf.config.list_physical_devices('GPU')

2021-08-31 11:40:16.872737: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 11:40:16.872761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 11:40:17.697069: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 11:40:17.697672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-31 11:40:17.769951: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-31 11:40:17.769974: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (helemanc-Latitude-5410): /pr

[]

# Load Dataframes

In [4]:
preprocess_path = "/home/helemanc/Desktop/Binary_Model/pre-processed"

TESS_df = pd.read_csv(os.path.join(preprocess_path,"TESS_df.csv"))

In [5]:
TESS_train = []
TESS_test = []

In [6]:
for index, row in TESS_df.iterrows(): 
    if row['actors'] == 'YAF': 
        TESS_train.append(row)
    else: 
        TESS_test.append(row)
len(TESS_train), len(TESS_test)

(1400, 1400)

In [9]:
TESS_df.head()

,emotion_label,gender,actors,path
0,fear,female,OAF,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
1,fear,female,OAF,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
2,fear,female,OAF,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
3,fear,female,OAF,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...
4,fear,female,OAF,/media/helemanc/OS/Users/i2CAT/Desktop/Dataset...


In [10]:
TESS_train = pd.DataFrame(TESS_train)
TESS_test = pd.DataFrame(TESS_test)

In [11]:
df_train = TESS_train

In [12]:
df_test = TESS_test

In [30]:
df_train = df_train.reset_index(drop=True)
df_train = df_test.reset_index(drop=True)

# Feature extraction

In [ ]:
df_train['emotion_label'].unique()

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_train['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_val['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
plt.title('Emotions distribution')
plt.hist(df_test['emotion_label'])
# plt.hist(y)
plt.show()

In [ ]:
df_train.shape

## Utils

In [41]:
def load_files(df):
    X = []
    for i in tqdm(df['path']): 
        X.append(librosa.load(i, res_type='kaiser_fast', sr=16000))
    return X

def extract_samples(X): 
    samples = []
    for ind,i in enumerate(X):
        samples.append(i[0])
    return samples 

def extract_labels(df): 
    labels = df['emotion_label'].copy()
    return labels 

def compute_lengths(samples): 
    lengths = [len(x) for x in samples]
    return lengths 

def check_outliers(lengths):
    # outliers
    lengths = np.array(lengths)
    print((lengths > 300000).sum())
    new_lengths = lengths[lengths < 300000]
    return new_lengths 

def compute_mean_length(lengths): 
    return lengths.mean()

def cut_and_pad(samples, labels, length_chosen = LENGTH_CHOSEN): 
    X_new = []
    y_new = []
    count = 0 
    for ind,i in enumerate(samples):
        if i.shape[0] < 300000:
            if i.shape[0] > length_chosen:
                new = i[:length_chosen]
                X_new.append(new)
            elif i.shape[0] < length_chosen:
                new = np.pad(i,math.ceil((length_chosen-i.shape[0])/2), mode='median')
                X_new.append(new)
            else:
                X_new.append(i)
            y_new.append(labels[count])
        count+=1
    
    return X_new, y_new

# Data Augmentation 
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# Data Augmentation 
def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)
    
def compute_mfccs(samples): 
    mfccs = []
    for i in tqdm(samples):
        mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=13)
        mfcc = mfcc.T
        mfcc = np.array(mfcc)
        mfccs.append(mfcc[:, 1:])
    mfccs = np.array(mfccs)
    return mfccs

def compute_mfccs_augmentation(samples, labels): 
    mfccs = []
    counter = 0 
    for i in tqdm(samples):

       # Weiner Filtering on original noise 
        samples_weiner = scipy.signal.wiener(i)
        is_fin = np.isfinite(samples_weiner).all()


        # Data Augmentation - Noise 
        noise_audio = noise(samples_weiner)

        # Data Augmentation - Pitch 
        pitch_audio = pitch(samples_weiner, sampling_rate=16000)


        # Data Augmentation -  pitch + noise 
        pn = pitch(noise_audio, sampling_rate = 16000)


        if is_fin: 
          # MFCC

          mfcc = librosa.feature.mfcc(y=i, sr=16000, n_mfcc=13)
          mfcc = mfcc.T
          mfccs.append(mfcc[:, 1:])

          mfcc_augmented = librosa.feature.mfcc(y=samples_weiner, sr=16000, n_mfcc=13)
          mfcc_augmented = mfcc_augmented.T
          mfccs.append(mfcc_augmented[:, 1:])

          mfcc_augmented_pitch = librosa.feature.mfcc(y=noise_audio, sr=16000, n_mfcc=13)
          mfcc_augmented_pitch = mfcc_augmented_pitch.T
          mfccs.append(mfcc_augmented_pitch[:, 1:])

          mfcc_augmented_p = librosa.feature.mfcc(y=pitch_audio, sr=16000, n_mfcc=13)
          mfcc_augmented_p = mfcc_augmented_p.T
          mfccs.append(mfcc_augmented_p[:, 1:]) 

          mfcc_augmented_pn = librosa.feature.mfcc(y=pn, sr=16000, n_mfcc=13)
          mfcc_augmented_pn = mfcc_augmented_pn.T
          mfccs.append(mfcc_augmented_pn[:, 1:]) 
    
    mfccs = np.array(mfccs)
    
    # Copy labels 
    y_prov = []
    y = labels 
    for i in range(len(y)): 
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
      y_prov.append(y[i])
    y = np.asarray(y_prov)

    return mfccs, y 





## Train

### Load samples and labels 

In [42]:
load_train = load_files(df_train)

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 2536.57it/s]


In [43]:
samples_train = extract_samples(load_train)

In [44]:
labels_train = extract_labels(df_train)

In [45]:
labels_train

0          fear
1          fear
2          fear
3          fear
4          fear
         ...   
1395    neutral
1396    neutral
1397    neutral
1398    neutral
1399    neutral
Name: emotion_label, Length: 1400, dtype: object

### Decide length

In [46]:
lengths = compute_lengths(samples_train) 

In [47]:
new_lengths = check_outliers(lengths) 

0


In [48]:
mean_length = compute_mean_length(new_lengths)

In [49]:
mean_length

32317.405714285713

### Cut and Pad

In [50]:
samples_train, labels_train = cut_and_pad(samples_train, labels_train)

In [51]:
samples_train = np.array(samples_train)
labels_train = np.array(labels_train)
print(samples_train.shape, labels_train.shape)

(1400,) (1400,)


### Feature Extraction - Without Data Augmentation 

In [52]:
mfccs_train = compute_mfccs(samples_train)

100%|███████████████████████████████████████| 1400/1400 [00:24<00:00, 57.99it/s]


In [53]:
mfccs_train.shape

(1400, 248, 12)

### Save features and labels

In [ ]:
'''
preprocess_path = "/home/helemanc/Desktop/Binary_Model/pre-processed"
mfccs_train_path = os.path.join(preprocess_path,"mfccs_train.pkl")
labels_train_path = os.path.join(preprocess_path,"labels_train.pkl")

with open(mfccs_train_path, 'wb') as f:
        pickle.dump(mfccs_train , f)
        
with open(labels_train_path, 'wb') as f:
        pickle.dump(labels_train , f)
'''

## Test

### Load samples and labels

In [54]:
load_test = load_files(df_test)

100%|█████████████████████████████████████| 1400/1400 [00:00<00:00, 2765.27it/s]


In [55]:
samples_test = extract_samples(load_test)

In [56]:
labels_test = extract_labels(df_test)

### Cut and Pad 

In [57]:
samples_test, labels_test = cut_and_pad(samples_test, labels_test)

In [58]:
samples_test = np.array(samples_test)
labels_test = np.array(labels_test)
print(samples_test.shape, labels_test.shape)

(1400,) (1400,)


### Feature Extraction

In [59]:
mfccs_test = compute_mfccs(samples_test)

100%|███████████████████████████████████████| 1400/1400 [00:24<00:00, 57.02it/s]


In [60]:
mfccs_test.shape

(1400, 248, 12)

# Encode Labels - Binary 

In [61]:
emotion_enc = {'fear':1, 'disgust':1, 'neutral':0, 'calm':0,  'happy':0, 'sadness':1, 'surprise':0, 'angry':1}

In [62]:
y_train = pd.Series(labels_train).replace(emotion_enc)
#y_val = pd.Series(labels_val).map(emotion_enc)
y_test = pd.Series(labels_test).map(emotion_enc)
#y_train_aug = pd.Series(labels_train_aug).map(emotion_enc)

# Train, Val, Test (X) 

In [63]:
X_train = mfccs_train
#X_val = mfccs_val
X_test = mfccs_test 

# Standard Scaling 

## Without Augmentation

In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
#X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

# Model 

In [ ]:
model = Sequential()

model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(248,12), kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.6))

model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64,  kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.summary()

# Training

## Without Augmentation

### Load Data

In [ ]:
'''
X_train = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_train.pkl", 'rb'))
X_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_val.pkl", 'rb'))
X_test = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/X_test.pkl", 'rb'))
y_train = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_train.pkl", 'rb'))
y_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_val.pkl", 'rb'))
y_test =  pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model/y_test.pkl", 'rb'))
'''

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [ ]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics='accuracy')

### Train

In [ ]:
history = model.fit(X_train, y_train, batch_size=4, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, model_checkpoint], class_weight = class_weights)

### Plot Training Graphs

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Save Model

In [ ]:
model.save("/home/helemanc/Desktop/Binary_Model/models/binary_model_ravdess_13_83")

### Evaluate Model

In [ ]:
model_loaded = tf.keras.models.load_model("/home/helemanc/Desktop/Binary_Model/models/binary_model")

In [ ]:
model.evaluate(X_test, y_test, batch_size=8)

In [ ]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

## With Augmentation

### Load Data

In [ ]:
'''
X_train_aug = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_train_aug.pkl", 'rb'))
X_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_val.pkl", 'rb'))
X_test = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/X_test.pkl", 'rb'))
y_train_aug = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_train_aug.pkl", 'rb'))
y_val = pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_val.pkl", 'rb'))
y_test =  pickle.load(open( "/home/helemanc/Desktop/Binary_Model/data_model_augmented/y_test.pkl", 'rb'))
'''

In [ ]:
X_train_aug = mfccs_train_aug
y_train_aug = labels_train_aug 

In [ ]:
from sklearn.utils import shuffle
X_train_aug, y_train_aug = shuffle(X_train_aug, y_train_aug)

In [ ]:
(unique, counts) = np.unique(y_train_aug, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_augmented.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_aug), y_train_aug)
class_weights = {l:c for l,c in zip(np.unique(y_train_aug), class_weights)}


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

### Train

In [ ]:
history = model.fit(X_train_aug, y_train_aug, batch_size=16, epochs=500, validation_data=(X_val, y_val),
           callbacks=[reduce_lr, early_stop, model_checkpoint], class_weight = class_weights)


### Plot Training Graphs

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### Save Model

In [ ]:
model.save("/home/helemanc/Desktop/Binary_Model/models/binary_model_aug")

### Evaluate Model

In [ ]:
model_loaded = tf.keras.models.load_model("/home/helemanc/Desktop/Binary_Model/models/binary_model_aug")

In [ ]:
model_loaded.evaluate(X_test, y_test, batch_size=16)

In [ ]:
from sklearn.metrics import classification_report
predictions = model_loaded.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

# Hyprparameter optimization 

In [ ]:
def create_model(optimizer='adam', init_mode='glorot_uniform'):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64,  kernel_initializer=tf.keras.initializers.HeNormal(seed=0)))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

## Initializer

In [ ]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
#import tensorflow as tf 
#from tf.keras.wrappers.scikit_learn import KerasClassifier

seed = 7
np.random.seed(seed)
batch_size = 4
epochs = 30

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']

param_grid = dict(init_mode=init_mode)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3), verbose = 10 )
grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)

In [ ]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

## Initializer, Batch Size, Learning Rate 

In [65]:
def create_model( init_mode='glorot_uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [66]:
weight_path = "/home/helemanc/Desktop/Binary_Model/weights/binary_model_l1l2.hdf5"

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                      save_weights_only=True, 
                                                      monitor='val_accuracy', 
                                                      mode='max', 
                                                      save_best_only=True)
# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}


In [67]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
#import tensorflow as tf 
#from tf.keras.wrappers.scikit_learn import KerasClassifier

seed = 7
np.random.seed(seed)
batch_size = 4
epochs = 50

model_CV = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'glorot_uniform', 'glorot_normal']
batches = [4,8,16]
lr = [0.001, 0.0001, 0.00005]

param_grid = dict(init_mode=init_mode, lr = lr, batch_size = batches)
grid = RandomizedSearchCV(estimator=model_CV, param_distributions=param_grid, n_jobs=-1, cv=KFold(3))
grid_result = grid.fit(X_train, y_train, callbacks=[reduce_lr, early_stop], class_weight = class_weights)

2021-08-31 11:47:30.445785: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 11:47:30.446312: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 11:47:30.454033: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-31 11:47:30.454061: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-08-31 11:47:30.473496: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

Epoch 1/50
59/59 - 8s - loss: 0.4063 - accuracy: 0.8167
Epoch 1/50
59/59 - 8s - loss: 0.8175 - accuracy: 0.5402
Epoch 1/50
59/59 - 8s - loss: 0.3840 - accuracy: 0.8382
Epoch 1/50
59/59 - 8s - loss: 0.6207 - accuracy: 0.7141
Epoch 1/50
59/59 - 8s - loss: 0.7483 - accuracy: 0.6206
Epoch 1/50
234/234 - 9s - loss: 0.4805 - accuracy: 0.7987
Epoch 1/50
234/234 - 9s - loss: 0.6045 - accuracy: 0.6699
Epoch 1/50
234/234 - 9s - loss: 0.5883 - accuracy: 0.6870
Epoch 2/50
59/59 - 7s - loss: 0.1117 - accuracy: 0.9593
Epoch 2/50
59/59 - 6s - loss: 0.6421 - accuracy: 0.6484
Epoch 2/50
59/59 - 6s - loss: 0.4582 - accuracy: 0.7976
Epoch 2/50
59/59 - 7s - loss: 0.0482 - accuracy: 0.9818
Epoch 2/50
59/59 - 7s - loss: 0.5937 - accuracy: 0.6699
Epoch 2/50
234/234 - 7s - loss: 0.2872 - accuracy: 0.9090
Epoch 2/50
234/234 - 7s - loss: 0.4636 - accuracy: 0.7728
Epoch 2/50
234/234 - 8s - loss: 0.3701 - accuracy: 0.8553
Epoch 3/50
59/59 - 7s - loss: 0.0620 - accuracy: 0.9829
Epoch 3/50
59/59 - 6s - loss: 0.3412

2021-08-31 12:10:39.350677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-31 12:10:39.351665: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-31 12:10:39.472191: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-31 12:10:39.491716: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299965000 Hz


Epoch 1/50
175/175 - 2s - loss: 0.5363 - accuracy: 0.8179
Epoch 2/50
175/175 - 1s - loss: 0.1819 - accuracy: 0.9407
Epoch 3/50
175/175 - 1s - loss: 0.1163 - accuracy: 0.9607
Epoch 4/50
175/175 - 1s - loss: 0.0725 - accuracy: 0.9750
Epoch 5/50
175/175 - 1s - loss: 0.0781 - accuracy: 0.9750
Epoch 6/50
175/175 - 1s - loss: 0.0595 - accuracy: 0.9779
Epoch 7/50
175/175 - 1s - loss: 0.0644 - accuracy: 0.9807
Epoch 8/50
175/175 - 1s - loss: 0.0431 - accuracy: 0.9871
Epoch 9/50
175/175 - 1s - loss: 0.0248 - accuracy: 0.9886
Epoch 10/50
175/175 - 1s - loss: 0.0566 - accuracy: 0.9857
Epoch 11/50
175/175 - 1s - loss: 0.0305 - accuracy: 0.9900
Epoch 12/50
175/175 - 1s - loss: 0.0210 - accuracy: 0.9943
Epoch 13/50
175/175 - 1s - loss: 0.0170 - accuracy: 0.9950
Epoch 14/50
175/175 - 1s - loss: 0.0058 - accuracy: 0.9971
Epoch 15/50
175/175 - 1s - loss: 0.0075 - accuracy: 0.9950
Epoch 16/50
175/175 - 1s - loss: 0.0132 - accuracy: 0.9957
Epoch 17/50
175/175 - 1s - loss: 0.0152 - accuracy: 0.9950
Epoch 

In [69]:
# print results
print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.6906271775563558 using {'lr': 0.001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.6813, std=0.188 using {'lr': 5e-05, 'init_mode': 'glorot_uniform', 'batch_size': 16}
 mean=0.6764, std=0.1987 using {'lr': 5e-05, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.6792, std=0.1995 using {'lr': 0.001, 'init_mode': 'uniform', 'batch_size': 16}
 mean=0.6799, std=0.2135 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 4}
 mean=0.6821, std=0.2079 using {'lr': 0.001, 'init_mode': 'glorot_uniform', 'batch_size': 8}
 mean=0.6778, std=0.1909 using {'lr': 0.0001, 'init_mode': 'uniform', 'batch_size': 8}
 mean=0.6721, std=0.1672 using {'lr': 0.001, 'init_mode': 'glorot_normal', 'batch_size': 4}
 mean=0.6906, std=0.1965 using {'lr': 0.001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
 mean=0.6635, std=0.1873 using {'lr': 0.0001, 'init_mode': 'glorot_normal', 'batch_size': 8}
 mean=0.6549, std=0.1623 using {'lr': 5e-05, 'init_mode': 'lecun_uniform', 'batch_size

## Train with best parameters

In [70]:
#Best Accuracy for 0.6906271775563558 using {'lr': 0.001, 'init_mode': 'lecun_uniform', 'batch_size': 8}
def create_model( init_mode='lecun_uniform', lr = 0.001):
    model = Sequential()

    model.add(layers.Conv1D(256, 5,padding='same',
                     input_shape=(248,12), kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.6))

    model.add(layers.Conv1D(128, 5,padding='same', kernel_initializer=init_mode))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=(4)))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, kernel_initializer=init_mode))
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    
    # compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer=Adam(lr = lr) , 
                  metrics=['accuracy'])
    return model

In [71]:
seed = 7
np.random.seed(seed)

In [72]:
model = create_model()

In [73]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [74]:
import datetime, os

In [75]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [76]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-08-31 12:22:00.379734: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-08-31 12:22:00.379817: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-08-31 12:22:00.437628: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


In [77]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', 
                                                 factor=0.5, patience=4, 
                                                 verbose=1, mode='max', 
                                                 min_lr=0.000001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=45, 
                                              verbose=1, restore_best_weights = True )

# classweight 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}

In [80]:
history = model.fit(X_train, y_train, batch_size=8, epochs=500,
           callbacks=[reduce_lr, early_stop, tensorboard_callback], class_weight = class_weights)

Epoch 1/500
 16/175 [=>............................] - ETA: 1s - loss: 4.1154 - accuracy: 0.8516

2021-08-31 12:23:49.486693: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-08-31 12:23:49.486720: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-08-31 12:23:49.505145: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2021-08-31 12:23:49.505823: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-08-31 12:23:49.507135: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210831-122200/train/plugins/profile/2021_08_31_12_23_49
2021-08-31 12:23:49.507809: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20210831-122200/train/plugins/profile/2021_08_31_12_23_49/helemanc-Latitude-5410.trace.json.gz
2021-08-31 12:23:49.508693: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20210831-122200/train/plugins/p

175/175 [==============================] - 1s 8ms/step - loss: 0.6789 - accuracy: 0.8993
Epoch 2/500
175/175 [==============================] - 1s 8ms/step - loss: 0.1561 - accuracy: 0.9429
Epoch 3/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0880 - accuracy: 0.9664
Epoch 4/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0878 - accuracy: 0.9714
Epoch 5/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0570 - accuracy: 0.9829
Epoch 6/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0526 - accuracy: 0.9793
Epoch 7/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0602 - accuracy: 0.9800
Epoch 8/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0402 - accuracy: 0.9814
Epoch 9/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0464 - accuracy: 0.9857
Epoch 10/500
175/175 [==============================] - 1s 8ms/step - loss: 0.0466 - accuracy: 0.9843
E

In [81]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 31441), started 1 day, 1:05:20 ago. (Use '!kill 31441' to kill it.)

In [82]:
model.evaluate(X_test, y_test, batch_size=8)

175/175 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 0.9993


[0.0014667337527498603, 0.9992856979370117]

In [83]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
pred = [1 * (x[0]>=0.50) for x in predictions] #0.5 o 0.52? 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       600
           1       1.00      1.00      1.00       800

    accuracy                           1.00      1400
   macro avg       1.00      1.00      1.00      1400
weighted avg       1.00      1.00      1.00      1400



## Save best model 

In [ ]:
model.save("/home/helemanc/Desktop/Binary_Model/models/binary_model_ravdess_13_after_optimization")